# Retrieval Augmented Generation (RAG)​ and ATT&CK Groups
References:
* https://python.langchain.com/en/latest/modules/indexes/getting_started.html

## Create ATT&CK Groups Knowledge Base

### Import Modules and Define Variables

In [1]:
from attackcti import attack_client
import os
import logging

logging.getLogger('taxii2client').setLevel(logging.CRITICAL)

In [2]:
# Define a few variables
current_directory = os.path.dirname("__file__")
knowledge_directory = os.path.join(current_directory, "knowledge")
db_directory = os.path.join(current_directory, "db")
templates_directory = os.path.join(current_directory, "templates")
group_template = os.path.join(templates_directory, "group.md")

### Initialize ATT&CK Client

In [3]:
lift = attack_client()

### Get ATT&CK Groups Knowledge
Gettings technique STIX objects used by all groups accross all ATT&CK matrices..

In [4]:
techniques_used_by_groups = lift.get_techniques_used_by_all_groups()
techniques_used_by_groups[0]

{'type': 'intrusion-set',
 'id': 'intrusion-set--b7f627e2-0817-4cd5-8d50-e75f8aa85cc6',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'created': '2023-02-23T15:31:38.829Z',
 'modified': '2023-04-17T21:49:16.371Z',
 'name': 'LuminousMoth',
 'description': '[LuminousMoth](https://attack.mitre.org/groups/G1014) is a Chinese-speaking cyber espionage group that has been active since at least October 2020. [LuminousMoth](https://attack.mitre.org/groups/G1014) has targeted high-profile organizations, including government entities, in Myanmar, the Philippines, Thailand, and other parts of Southeast Asia. Some security researchers have concluded there is a connection between [LuminousMoth](https://attack.mitre.org/groups/G1014) and [Mustang Panda](https://attack.mitre.org/groups/G0129) based on similar targeting and TTPs, as well as network infrastructure overlaps.(Citation: Kaspersky LuminousMoth July 2021)(Citation: Bitdefender LuminousMoth July 2021)',
 'aliases': ['L

### Create ATT&CK Groups Documents

In [5]:
import copy
from jinja2 import Template

# Create Group docs
all_groups = dict()
for technique in techniques_used_by_groups:
    if technique['id'] not in all_groups:
        group = dict()
        group['group_name'] = technique['name']
        group['group_id'] = technique['external_references'][0]['external_id']
        group['created'] = technique['created']
        group['modified'] = technique['modified']
        group['description'] = technique['description']
        group['aliases'] = technique['aliases']
        if 'x_mitre_contributors' in technique:
            group['contributors'] = technique['x_mitre_contributors']
        group['techniques'] = []
        all_groups[technique['id']] = group
    technique_used = dict()
    technique_used['matrix'] = technique['matrix']
    technique_used['domain'] = technique['x_mitre_domains']
    technique_used['platform'] = technique['platform']
    technique_used['tactics'] = technique['tactic']
    technique_used['technique_id'] = technique['technique_id']
    technique_used['technique_name'] = technique['technique']
    technique_used['use'] = technique['relationship_description']
    if 'data_sources' in technique:
        technique_used['data_sources'] = technique['data_sources']
    all_groups[technique['id']]['techniques'].append(technique_used)

if not os.path.exists(knowledge_directory):
   print("[+] Creating knowledge directory..")
   os.makedirs(knowledge_directory)

print("[+] Creating markadown files for each group..")
markdown_template = Template(open(group_template).read())
for key in list(all_groups.keys()):
    group = all_groups[key]
    print("  [>>] Creating markdown file for {}..".format(group['group_name']))
    group_for_render = copy.deepcopy(group)
    markdown = markdown_template.render(metadata=group_for_render, group_name=group['group_name'], group_id=group['group_id'])
    file_name = (group['group_name']).replace(' ','_')
    open(f'{knowledge_directory}/{file_name}.md', encoding='utf-8', mode='w').write(markdown)

[+] Creating markadown files for each group..
  [>>] Creating markdown file for LuminousMoth..
  [>>] Creating markdown file for Metador..
  [>>] Creating markdown file for CURIUM..
  [>>] Creating markdown file for EXOTIC LILY..
  [>>] Creating markdown file for Moses Staff..
  [>>] Creating markdown file for SideCopy..
  [>>] Creating markdown file for Aoqin Dragon..
  [>>] Creating markdown file for Earth Lusca..
  [>>] Creating markdown file for POLONIUM..
  [>>] Creating markdown file for LAPSUS$..
  [>>] Creating markdown file for Ember Bear..
  [>>] Creating markdown file for BITTER..
  [>>] Creating markdown file for Aquatic Panda..
  [>>] Creating markdown file for Confucius..
  [>>] Creating markdown file for LazyScripter..
  [>>] Creating markdown file for TeamTNT..
  [>>] Creating markdown file for Andariel..
  [>>] Creating markdown file for Ferocious Kitten..
  [>>] Creating markdown file for IndigoZebra..
  [>>] Creating markdown file for BackdoorDiplomacy..
  [>>] Creat

## Generate Knowledge Base Embeddings

### Load Documents

In [6]:
import glob
from langchain.document_loaders import UnstructuredMarkdownLoader

In [7]:
# variables
group_files = glob.glob(os.path.join(knowledge_directory, "*.md"))

# Loading Markdown files
md_docs = []
print("[+] Loading Group markdown files..")
for group in group_files:
    print(f' [*] Loading {os.path.basename(group)}')
    loader = UnstructuredMarkdownLoader(group)
    md_docs.extend(loader.load())

print(f'[+] Number of .md documents processed: {len(md_docs)}')

[+] Loading Group markdown files..
 [*] Loading admin@338.md
 [*] Loading Ajax_Security_Team.md
 [*] Loading ALLANITE.md
 [*] Loading Andariel.md
 [*] Loading Aoqin_Dragon.md
 [*] Loading APT-C-36.md
 [*] Loading APT1.md
 [*] Loading APT12.md
 [*] Loading APT16.md
 [*] Loading APT17.md
 [*] Loading APT18.md
 [*] Loading APT19.md
 [*] Loading APT28.md
 [*] Loading APT29.md
 [*] Loading APT3.md
 [*] Loading APT30.md
 [*] Loading APT32.md
 [*] Loading APT33.md
 [*] Loading APT37.md
 [*] Loading APT38.md
 [*] Loading APT39.md
 [*] Loading APT41.md
 [*] Loading Aquatic_Panda.md
 [*] Loading Axiom.md
 [*] Loading BackdoorDiplomacy.md
 [*] Loading BITTER.md
 [*] Loading BlackOasis.md
 [*] Loading BlackTech.md
 [*] Loading Blue_Mockingbird.md
 [*] Loading BRONZE_BUTLER.md
 [*] Loading Carbanak.md
 [*] Loading Chimera.md
 [*] Loading Cleaver.md
 [*] Loading Cobalt_Group.md
 [*] Loading Confucius.md
 [*] Loading CopyKittens.md
 [*] Loading CURIUM.md
 [*] Loading Darkhotel.md
 [*] Loading DarkHyd

Check a doc page content

In [8]:
print(md_docs[0].page_content)

admin@338 - G0018

Created: 2017-05-31T21:31:53.579Z

Modified: 2020-03-18T19:54:59.120Z

Contributors: Tatsuya Daitoku, Cyber Defense Institute, Inc.

Aliases

admin@338

Description

admin@338 is a China-based cyber threat group. It has previously used newsworthy events as lures to deliver malware and has primarily targeted organizations involved in financial, economic, and trade policy, typically using publicly available RATs such as PoisonIvy, as well as some non-public backdoors. (Citation: FireEye admin@338)

Techniques Used

admin@338 has sent emails with malicious Microsoft Office documents attached.(Citation: FireEye admin@338)|
|mitre-attack|enterprise-attack|Linux,macOS,Windows|T1204.002|Malicious File|

admin@338 has attempted to get victims to launch malicious Microsoft Word attachments delivered via spearphishing emails.(Citation: FireEye admin@338)|
|mitre-attack|enterprise-attack|Linux,Windows,macOS|T1203|Exploitation for Client Execution|

admin@338 has exploited clien

Check token counts on loaded documents

In [9]:
import tiktoken

tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')
token_integers = tokenizer.encode(md_docs[0].page_content, disallowed_special=())
num_tokens = len(token_integers)
token_bytes = [tokenizer.decode_single_token_bytes(token) for token in token_integers]
print()
print(f"token count: {num_tokens} tokens")
print(f"token integers: {token_integers}")
print(f"token bytes: {token_bytes}")


token count: 532 tokens
token integers: [2953, 31, 18633, 482, 480, 4119, 23, 271, 11956, 25, 220, 679, 22, 12, 2304, 12, 2148, 51, 1691, 25, 2148, 25, 4331, 13, 24847, 57, 271, 19696, 25, 220, 2366, 15, 12, 2839, 12, 972, 51, 777, 25, 4370, 25, 2946, 13, 4364, 57, 271, 54084, 9663, 25, 350, 1900, 45644, 423, 1339, 16900, 11, 34711, 16777, 10181, 11, 4953, 382, 96309, 271, 2953, 31, 18633, 271, 5116, 271, 2953, 31, 18633, 374, 264, 5734, 6108, 21516, 6023, 1912, 13, 1102, 706, 8767, 1511, 502, 2332, 34594, 4455, 439, 326, 1439, 311, 6493, 40831, 323, 706, 15871, 17550, 11351, 6532, 304, 6020, 11, 7100, 11, 323, 6696, 4947, 11, 11383, 1701, 17880, 2561, 98980, 82, 1778, 439, 52212, 40, 14029, 11, 439, 1664, 439, 1063, 2536, 57571, 1203, 28404, 13, 320, 34, 7709, 25, 6785, 51158, 4074, 31, 18633, 696, 29356, 8467, 12477, 271, 2953, 31, 18633, 706, 3288, 14633, 449, 39270, 5210, 8410, 9477, 12673, 13127, 34, 7709, 25, 6785, 51158, 4074, 31, 18633, 8, 7511, 91, 1800, 265, 12, 21208, 91, 7

In [10]:
# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=() #To disable this check for all special tokens
    )
    return len(tokens)

# Get token counts
token_counts = [tiktoken_len(doc.page_content) for doc in md_docs]

print(f"""[+] Token Counts:
Min: {min(token_counts)}
Avg: {int(sum(token_counts) / len(token_counts))}
Max: {max(token_counts)}""")

[+] Token Counts:
Min: 176
Avg: 1619
Max: 7346


### Split Documents

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
# Chunking Text
print('[+] Initializing RecursiveCharacterTextSplitter..')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

[+] Initializing RecursiveCharacterTextSplitter..


In [13]:
print('[+] Splitting documents in chunks..')
chunks = text_splitter.split_documents(md_docs)

print(f'[+] Number of documents: {len(md_docs)}')
print(f'[+] Number of chunks: {len(chunks)}')

[+] Splitting documents in chunks..
[+] Number of documents: 134
[+] Number of chunks: 534


In [14]:
print(chunks[0])
print(chunks[1])

page_content='admin@338 - G0018\n\nCreated: 2017-05-31T21:31:53.579Z\n\nModified: 2020-03-18T19:54:59.120Z\n\nContributors: Tatsuya Daitoku, Cyber Defense Institute, Inc.\n\nAliases\n\nadmin@338\n\nDescription\n\nadmin@338 is a China-based cyber threat group. It has previously used newsworthy events as lures to deliver malware and has primarily targeted organizations involved in financial, economic, and trade policy, typically using publicly available RATs such as PoisonIvy, as well as some non-public backdoors. (Citation: FireEye admin@338)\n\nTechniques Used\n\nadmin@338 has sent emails with malicious Microsoft Office documents attached.(Citation: FireEye admin@338)|\n|mitre-attack|enterprise-attack|Linux,macOS,Windows|T1204.002|Malicious File|\n\nadmin@338 has attempted to get victims to launch malicious Microsoft Word attachments delivered via spearphishing emails.(Citation: FireEye admin@338)|\n|mitre-attack|enterprise-attack|Linux,Windows,macOS|T1203|Exploitation for Client Execu

We can contribute this!

In [17]:
import hashlib

json_documents = []
m = hashlib.md5()
for doc in md_docs:
    doc_name = os.path.basename(doc.metadata['source'])
    m.update(doc_name.encode('utf-8'))
    uid = m.hexdigest()[:12]
    chunks_strings = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks_strings):
        # Add JSON object to array
        json_documents.append({
            'id': f'{uid}-{i}',
            'text': chunk,
            'source': doc_name
        })

In [18]:
print(json_documents[0])
print(json_documents[1])

{'id': '4d1ab63e9fd8-0', 'text': 'admin@338 - G0018\n\nCreated: 2017-05-31T21:31:53.579Z\n\nModified: 2020-03-18T19:54:59.120Z\n\nContributors: Tatsuya Daitoku, Cyber Defense Institute, Inc.\n\nAliases\n\nadmin@338\n\nDescription\n\nadmin@338 is a China-based cyber threat group. It has previously used newsworthy events as lures to deliver malware and has primarily targeted organizations involved in financial, economic, and trade policy, typically using publicly available RATs such as PoisonIvy, as well as some non-public backdoors. (Citation: FireEye admin@338)\n\nTechniques Used\n\nadmin@338 has sent emails with malicious Microsoft Office documents attached.(Citation: FireEye admin@338)|\n|mitre-attack|enterprise-attack|Linux,macOS,Windows|T1204.002|Malicious File|\n\nadmin@338 has attempted to get victims to launch malicious Microsoft Word attachments delivered via spearphishing emails.(Citation: FireEye admin@338)|\n|mitre-attack|enterprise-attack|Linux,Windows,macOS|T1203|Exploitat

### Export Knowledge Base as JSONL File (Optional)

In [19]:
import json

if not os.path.exists(db_directory):
   print('[+] Creating database directory..')
   os.makedirs(db_directory)

print(f'[+] Exporting groups as .jsonl file..')
with open(f'{os.path.join(db_directory, "attack-groups.jsonl")}', 'w') as f:
    for doc in json_documents:
        f.write(json.dumps(doc) + '\n')

[+] Exporting groups as .jsonl file..


### Generate Embeddings

In [52]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import openai
import os
from dotenv import load_dotenv

In [53]:
# Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [65]:
print("[+] Starting embedding..")
embeddings = OpenAIEmbeddings()

# Send text chunks to OpenAI Embeddings API
print("[+] Sending chunks to OpenAI Embeddings API..")
db = FAISS.from_documents(chunks, embeddings)

[+] Starting embedding..
[+] Sending chunks to OpenAI Embeddings API..


### Create Database Pickle File (Optional)

In [67]:
import pickle
import os

current_directory = os.path.dirname("__file__")
db_directory = os.path.join(current_directory, "db")
db_file = os.path.join(db_directory,"db.pkl")

# Save vectorstore
print("[+] Create Pickle file..")
with open(db_file, "wb") as f:
    pickle.dump(db, f)

[+] Create Pickle file..


You can then load it like this

In [21]:
import pickle

db_file = os.path.join(db_directory,"db.pkl")

if os.path.exists(db_file):
    with open(db_file, "rb") as f:
        db = pickle.load(f)

print(type(db))

<class 'langchain.vectorstores.faiss.FAISS'>


## Query ATT&CK Groups Knowledge Base

### Initialize OpenAI (Optional)

In [22]:
from dotenv import load_dotenv
import openai

# Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

### Define Vector Store Retriever
The retriever interface is a generic interface that makes it easy to combine documents with language models. This interface exposes a get_relevant_documents method which takes in a query (a string) and returns a list of documents.

In [30]:
retriever = db.as_retriever(search_kwargs={"k":5})

### Get Relevant Documents

#### Phishings Techniques used by Threat Actors

In [70]:
query = "What are some phishing techniques used by threat actors?"

In [71]:
print("[+] Getting relevant documents for query..")
relevant_docs = retriever.get_relevant_documents(query)
relevant_docs

[+] Getting relevant documents for query..


[Document(page_content='TA505 has used spearphishing emails with malicious attachments to initially compromise victims.(Citation: Proofpoint TA505 Sep 2017)(Citation: Proofpoint TA505 June 2018)(Citation: Proofpoint TA505 Jan 2019)(Citation: Cybereason TA505 April 2019)(Citation: ProofPoint SettingContent-ms July 2018)(Citation: Proofpoint TA505 Mar 2018)(Citation: Trend Micro TA505 June 2019)(Citation: Proofpoint TA505 October 2019)(Citation: IBM TA505 April 2020)|', metadata={'source': 'knowledge\\TA505.md'}),
 Document(page_content='Nomadic Octopus has targeted victims with spearphishing emails containing malicious attachments.(Citation: Security Affairs DustSquad Oct 2018)(Citation: ESET Nomadic Octopus 2018)|', metadata={'source': 'knowledge\\Nomadic_Octopus.md'}),
 Document(page_content="APT33 utilized PowerShell scripts to establish command and control and install files for execution. (Citation: Symantec March 2019) (Citation: Dragos)|\n|mitre-attack|enterprise-attack,ics-attack

#### Question Answering

In [28]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [73]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
chain.run(input_documents=relevant_docs, question=query)

' Threat actors have used spearphishing emails with malicious attachments, malicious links, password-protected RAR archives containing malicious Excel Web Query files (.iqy), malicious Microsoft Office documents that use the “attachedTemplate” technique to load a template from a remote server, links to HTML application files embedded with malicious code, and SMS and email messages with links designed to steal credentials or track victims.'

#### Cloud Administration Command

In [34]:
query = "Tell me more about technique 'Cloud Administration Command' (T1651)."

In [35]:
print("[+] Getting relevant documents for query..")
relevant_docs = retriever.get_relevant_documents(query)
relevant_docs

[+] Getting relevant documents for query..


[Document(page_content='TA505 has used TinyMet to enumerate members of privileged groups.(Citation: IBM TA505 April 2020)\n\nTA505 has also run\n\nTA505 has used the tool EmailStealer to steal and send lists of e-mail addresses to a remote server.(Citation: Trend Micro TA505 June 2019)|\n|mitre-attack|enterprise-attack|Linux,macOS,Windows|T1568.001|Fast Flux DNS|\n\nTA505 has used fast flux to mask botnets by distributing payloads across multiple IPs.(Citation: Trend Micro TA505 June 2019)|\n|mitre-attack|enterprise-attack|Windows|T1059.003|Windows Command Shell|\n\nTA505 has executed commands using\n\nTA505 has used JavaScript for code execution.(Citation: Proofpoint TA505 Sep 2017)(Citation: Proofpoint TA505 June 2018)|\n|mitre-attack|enterprise-attack|Linux,macOS,Windows|T1204.001|Malicious Link|\n\nTA505 has used lures to get users to click links in emails and attachments. For example,\n\nTA505 makes their malware look like legitimate Microsoft Word documents, .pdf and/or .lnk file

In [36]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
chain.run(input_documents=relevant_docs, question=query)

' T1651 Cloud Administration Command is a technique used by threat actors to gain access to cloud-based services and resources. This technique is often used in conjunction with other techniques such as credential dumping, spear phishing, and exploitation of cloud-based services. It can be used to gain access to cloud-based resources, such as virtual machines, storage, and databases, as well as to execute commands on cloud-based systems.'